# Neuromodulation example  (bath) -- Equilibrium simulation

This neuromodulation creates a small network of neurons. We also use the reaction diffusion model by Anu G Nair 2015.

To generate the ```reaction_diffusion_D1.json``` file from the xml file we run ```data/convert_sbml_to_json.sh```

Reference for DA modulation cascade:

Lindroos R, Dorst MC, Du K, et al. Basal Ganglia Neuromodulation Over Multiple Temporal and Structural Scales-Simulations of Direct Pathway MSNs Investigate the Fast Onset of Dopaminergic Effects and Predict the Role of Kv4.2. Front Neural Circuits. 2018;12:3. Published 2018 Feb 6. doi:10.3389/fncir.2018.00003

Here we use the data from Planert:
Planert H, Berger TK, Silberberg G. Membrane properties of striatal direct and indirect pathway neurons in mouse and rat slices and their modulation by dopamine. PLoS One. 2013;8(3):e57054. doi:10.1371/journal.pone.0057054

The neurons are held at -60mV, or -80mV, then a current injection is applied for 500ms to trigger spiking.

Figure 4:



## Network setup

We have two neurons. The first neuron (Neuron 0) receives external input (cortical from t=0s and DA from t=0.1s). The cortical input will activate the first neuron, and through activation of synapses on the second neuron, we will see the dopamine level increase in the second neuron (Neuron 1).

The first neuron also receives direct DA activation from external input (starting at 100ms).

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import plotly.io as pio 
pio.templates.default = "simple_white"

In [3]:
import os
from snudda import Snudda

neuron_type = "dSPN"
neuron_path = os.path.join("data", neuron_type.lower())

network_path = os.path.join("networks", "Bevan2020_neuromodulation_bath_current_equilibrium_SBML")

In [4]:
snudda = Snudda(network_path=network_path)
#si = snudda.init_tiny(neuron_paths=neuron_path, neuron_names="ispn", number_of_neurons=[1], 
#                      random_seed=123)
si = snudda.init_tiny(neuron_paths=neuron_path, neuron_names=neuron_type, number_of_neurons=[1], 
                      random_seed=123)

if neuron_type == "dSPN":
    reaction_diffusion_file = "data/JSON/reaction_diffusion_D1.json"
elif neuron_type == "iSPN":
    reaction_diffusion_file = "data/JSON/reaction_diffusion_D2.json"


si.network_data["regions"]["Cube"]["neurons"][neuron_type]["reaction_diffusion"] = reaction_diffusion_file

# How the ion channels are modified by DA
si.network_data["regions"]["Cube"]["neurons"][neuron_type]["modulation"] = "test-modulation.json"
si.network_data["regions"]["Cube"]["neurons"][neuron_type]["modulation_key"] = "abc"

si.write_json()

snudda.create_network()

Warning, no snudda_data set!
Adding neurons: dSPN from dir data/dspn
Writing networks/Bevan2020_neuromodulation_bath_current_equilibrium_SBML/network-config.json
Writing networks/Bevan2020_neuromodulation_bath_current_equilibrium_SBML/network-config.json
Placing neurons
Network path: networks/Bevan2020_neuromodulation_bath_current_equilibrium_SBML
Creating missing directory networks/Bevan2020_neuromodulation_bath_current_equilibrium_SBML/log
Created directory networks/Bevan2020_neuromodulation_bath_current_equilibrium_SBML/log
Reading SNUDDA_DATA=None from networks/Bevan2020_neuromodulation_bath_current_equilibrium_SBML/network-config.json
No n_putative_points and putative_density, setting n_putative_points = 46
(this must be larger than the number of neurons you want to place)
Generating 46 points for networks/Bevan2020_neuromodulation_bath_current_equilibrium_SBML/mesh/Cube-cube-mesh-2.3159794767993218e-05.obj
Filtering, keeping inside points: 1 / 20
Managed to create 1 putative poin

In [5]:
# Free memory
snudda = None

### No synaptic input, only current injections!

In [6]:
sim_output_neuromodulation_ON = os.path.join(network_path, "simulation", "output_neuromodulation_ON.hdf5")
sim_output_neuromodulation_OFF = os.path.join(network_path, "simulation", "output_neuromodulation_OFF.hdf5")

In [7]:
# mech_dir = os.path.join("..", "..", "..", "..", "BasalGangliaData", "data", "neurons", "mechanisms")
mech_dir = "/home/hjorth/BasalGangliaData/data/neurons/mechanisms"
sample_dt = None # 0.00005

# sim = snudda.simulate(time=0, mech_dir=mech_dir, verbose=True, sample_dt=sample_dt, output_file=sim_output_neuromodulation_ON)

In [8]:
# sim_config_on = os.path.join("data", "da_experiment_on.json")
# sim_config_off = os.path.join("data", "da_experiment_off.json")

snudda = None

# sim_config_on = os.path.join("data", "da_experiment_cur_inj_on_bath_sbtab.json")
sim_config_on = os.path.join("data", "da_experiment_cur_inj_on_bath-find-equilibrium_SBML-BevanDA.json")
# sim_config_off = os.path.join("data", "da_experiment_cur_inj_off_bath_sbtab.json")


sim_time = 1000
n_workers = 1  # 2

## Running simulations

To see progress of the two simulations in log files ```networks/neuromodulation_ON_OFF/log/network-simulation-ON.txt``` and ```networks/neuromodulation_ON_OFF/log/network-simulation-OFF.txt```.

In [9]:
run_str_on = f"snudda simulate {network_path} --time {sim_time} --simulation_config {sim_config_on} --mechdir {mech_dir} --enable_rxd_neuromodulation"
print(run_str_on)

snudda simulate networks/Bevan2020_neuromodulation_bath_current_equilibrium_SBML --time 1000 --simulation_config data/da_experiment_cur_inj_on_bath-find-equilibrium_SBML-BevanDA.json --mechdir /home/hjorth/BasalGangliaData/data/neurons/mechanisms --enable_rxd_neuromodulation


In [10]:
# %%timeit
os.system(run_str_on)

numprocs=1
args.ipython_profile = None
args: Namespace(action='simulate', path='networks/Bevan2020_neuromodulation_bath_current_equilibrium_SBML', network_file=None, input_file=None, output_file=None, time=1000.0, snudda_data=None, simulation_config='data/da_experiment_cur_inj_on_bath-find-equilibrium_SBML-BevanDA.json', record_volt=True, randomseed=None, disable_synapses=None, disable_gj=None, mech_dir='/home/hjorth/BasalGangliaData/data/neurons/mechanisms', profile=False, verbose=False, exportCoreNeuron=False, record_all=None, enable_rxd_neuromodulation=True, disable_rxd_neuromodulation=None, ipython_profile=None)
MPI Rank: 0, Size: 1
Creating missing directory networks/Bevan2020_neuromodulation_bath_current_equilibrium_SBML/simulation
Created directory networks/Bevan2020_neuromodulation_bath_current_equilibrium_SBML/simulation
Using input file None
NEURON mechanisms already compiled, make sure you have the correct version of NEURON modules.
If you delete x86_64, aarch64, arm64 direc

0

## Load the data and plot

In [11]:
from snudda.utils import SnuddaLoadSimulation

nd = SnuddaLoadSimulation(sim_output_neuromodulation_ON)
time = nd.get_time()
data_pka = nd.get_data("PKAc", 0)[0][0]
data_da = nd.get_data("DA", 0)[0][0]
data_da_external = nd.get_data("DA", 0)[0][0]

# This is saved with add_rxd_internal_concentration_recording_all -- check that it worked 
data_pka_all0 = nd.get_data("PKAc", 0)[0][0]

Loading networks/Bevan2020_neuromodulation_bath_current_equilibrium_SBML/simulation/output_neuromodulation_ON.hdf5
WARNING. Depolarisation block in neuron - neuron_id: (name, parameter_key, morphology_key):
0: (dSPN_0, pe6ec2d4b, mf702205f)


In [12]:
nd.list_data_types(0)

['AC5',
 'AC5GaolfGTP',
 'AC5GaolfGTP_ATP',
 'AC5_ATP',
 'AMP',
 'ATP',
 'D1R',
 'D1RDA',
 'D1RDAGolf',
 'D1RGolf',
 'DA',
 'GaolfGDP',
 'GaolfGTP',
 'Gbgolf',
 'Golf',
 'PDE10',
 'PDE10_cAMP',
 'PDE10c',
 'PDE10c_cAMP',
 'PDE4',
 'PDE4_cAMP',
 'PKA',
 'PKAc',
 'PKAcAMP2',
 'PKAcAMP4',
 'PKAreg',
 'cAMP',
 'kaf_ms.modulation_factor_g',
 'kaf_ms.modulation_factor_shift',
 'kir_ms.modulation_factor',
 'naf_ms.modulation_factor',
 'spikes',
 'voltage']

In [13]:
data_types = nd.list_data_types(0)
all_species_data = nd.get_all_data(neuron_id=0, exclude=["spikes", "voltage"])
time = nd.get_time()
voltage = nd.get_data("voltage", [0])

In [14]:
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "iframe"  # Do not save plots in the notebook, they can get BIG

fig = go.Figure()
for data_type in all_species_data:
    idx = time >= 0.0
    fig.add_trace(go.Scatter(x=time[idx], y=all_species_data[data_type][0][0].T[0][idx], name=data_type, line={"width":4}))

"""
fig.update_layout(xaxis_title="Time (s)", yaxis_title="Concentration", width=1000, height=800,
                 font={"size":18},  # General font size for all elements
                 legend={"font":{"size":16}},  # Specific font size for legend
                 xaxis={"title":{"font":{"size":20}}, "tickfont":{"size":14}},  # X-axis title and tick labels
                 yaxis={"title":{"font":{"size":20}}, "tickfont":{"size":14}})   # Y-axis title and tick labels
"""
fig.show()

# Extracting steady state values

Extract steady state values: 
```python ../../../snudda/utils/extract_steady_state_values.py data/JSON/reaction_diffusion_D1.json networks/Bevan2020_neuromodulation_bath_current_equilibrium_SBML/simulation/output_neuromodulation_ON.hdf5```

Verify that the rates are correct, the rename the JSON file ```reaction_diffusion_D1.json-updated``` to ```reaction_diffusion_D1_Bevan2020.json```